In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import signal

# When Models hurt prediction : a DC motor trial (p.132/155)

## Définition du modèle et simulation

Modèle sous forme d'espace d'état :

$\begin{cases}
\frac{d}{dt}\begin{bmatrix}i(t)\\
\omega_{R}(t)
\end{bmatrix} & =\begin{bmatrix}-\frac{R}{L} & -\frac{K}{L}\\
\frac{K}{J} & -\frac{f}{J}
\end{bmatrix}\begin{bmatrix}i(t)\\
\omega_{R}(t)
\end{bmatrix}+\begin{bmatrix}\frac{1}{L} & 0\\
0 & -\frac{1}{J}
\end{bmatrix}\begin{bmatrix}V_{S}(t)\\
\tau_{L}(t)
\end{bmatrix}\\
\begin{bmatrix}i_{obs}(t)\\
\omega_{Robs}(t)\\
V_{Lobs}(t)
\end{bmatrix} & =\begin{bmatrix}1 & 0\\
0 & 1\\
-R & -K
\end{bmatrix}\begin{bmatrix}i(t)\\
\omega_{R}(t)
\end{bmatrix}+\begin{bmatrix}0 & 0\\
0 & 0\\
1 & 0
\end{bmatrix}\begin{bmatrix}V_{S}(t)\\
\tau_{L}(t)
\end{bmatrix}
\end{cases}$

In [2]:
def heaviside(x):
    if x == 0:
        return 0.5

    return 0 if x < 0 else 1

## Simulation intéractive

In [60]:
@interact
def _(x0 = slider(0,10,10/100.0),
    x1 = slider(0,1,1/100.0),
    sim_duration = slider(10,30,default=10,label="Simulation duration"),
    sim_nb_samples = slider(10,100, default=100,label="Samples number"),
    steptime = slider(0,10,default=5,label='Step time'),
    stepgain = slider(-5,5,default=2,label='Step gain'),
    R = slider(0,2,2/100.0,default=1,label='R'),
    K = slider(0,0.02,0.02/100.0,default=0.01),
    L = slider(0.5/100,1,1/100.0,default=0.5),
    J = slider(0.01/100.0,0.02,0.02/100.0,default=0.01),
    f = slider(0,0.2,0.2/100.0,default=0.1)):

    A = matrix([[ -R/L, -K/L ],
            [ K/J , -f/J ]])
    
    B = matrix([[ 1/L , 0    ],
                [ 0   , -1/J ]])

    C = matrix([[ 1 , 0  ],
                [ 0 , 1  ],
                [ -R, -K ]])

    D = matrix([[ 0, 0 ],
                [ 0, 0 ],
                [ 1, 0 ]])

    ss_mcc = signal.lti(A,B,C,D)
    
    t = np.linspace(0,sim_duration,sim_nb_samples)
    u1 = [stepgain*heaviside(x-steptime) for x in np.nditer(t)]
    u2 = np.array(np.zeros_like(t))
    U=np.vstack((u1,u2)).transpose()
    X0 = (x0,x1)
    tout, Y, X = signal.lsim(ss_mcc, U, t, X0)

    # Plot help  : http://sage.brandoncurtis.com/plotting.html
    # Plot help2 : http://doc.sagemath.org/html/en/reference/plotting/sage/plot/plot.html
    # Matplotlib examples : http://matplotlib.org/examples/index.html
    
    f, axarr = plt.subplots(3, sharex=True, figsize=(10,6))
    
    axarr[0].plot(t, Y[:,0], 'b')
    axarr[0].grid(True)
    axarr[0].set_ylabel('$i_{obs}(t)$',fontsize=20)

    axarr[1].plot(t, Y[:,1], 'r')
    axarr[1].grid(True)
    axarr[1].set_ylabel('$\omega_{R_{obs}}(t)$',fontsize=20)
    
    axarr[2].plot(t, Y[:,2], 'g')
    axarr[2].grid(True)
    axarr[2].set_ylabel('$V_{L_{obs}}(t)$',fontsize=20)
    
    plt.xlabel('Time (s)',fontsize=16)
    
    f.subplots_adjust(hspace=0.3)
    
    plt.show()

## RRA

In [61]:
@interact
def _(x0 = slider(0,10,10/100.0),
    x1 = slider(0,1,1/100.0),
    sim_duration = slider(10,30,default=10,label="Simulation duration"),
    sim_nb_samples = slider(10,100, default=100,label="Samples number"),
    steptime = slider(0,10,default=5,label='Step time'),
    stepgain = slider(-5,5,default=2,label='Step gain'),
    R = slider(0,2,2/100.0,default=1,label='R'),
    K = slider(0,0.02,0.02/100.0,default=0.01),
    L = slider(0.5/100,1,1/100.0,default=0.5),
    J = slider(0.01/100.0,0.02,0.02/100.0,default=0.01),
    f = slider(0,0.2,0.2/100.0,default=0.1)):

    ##### Modèle oracle
    A = matrix([[ -R/L, -K/L ],
            [ K/J , -f/J ]])
    
    B = matrix([[ 1/L , 0    ],
                [ 0   , -1/J ]])

    C = matrix([[ 1 , 0  ],
                [ 0 , 1  ],
                [ -R, -K ]])

    D = matrix([[ 0, 0 ],
                [ 0, 0 ],
                [ 1, 0 ]])

    ##### Modèle défectueux
    Af = matrix([[ -R*1.3/L, -K/L+1 ],
            [ K/J , -f/J ]])
    
    Bf = matrix([[ 1.2/L , 0    ],
                [ 0   , -1/J ]])

    Cf = matrix([[ 1 , 0  ],
                [ 0 , 1  ],
                [ -R, -K ]])

    Df = matrix([[ 0, 0 ],
                [ 0, 0 ],
                [ 1, 0 ]])
    
    ss_mcc = signal.lti(A,B,C,D)
    ss_faulty_mcc = signal.lti(Af,Bf,Cf,Df)
    
    t = np.linspace(0,sim_duration,sim_nb_samples)
    u1 = [stepgain*heaviside(x-steptime) for x in np.nditer(t)]
    u2 = np.array(np.zeros_like(t))
    U=np.vstack((u1,u2)).transpose()
    X0 = (x0,x1)
    tout, Y, X = signal.lsim(ss_mcc, U, t, X0)
    tout, Yobs, Xobs = signal.lsim(ss_faulty_mcc, U, t, X0)

    # Plot help  : http://sage.brandoncurtis.com/plotting.html
    # Plot help2 : http://doc.sagemath.org/html/en/reference/plotting/sage/plot/plot.html
    # Matplotlib examples : http://matplotlib.org/examples/index.html
    
    var('i_obs,omega_R_obs,V_L_obs,V_S')
    r_1 = [(R*i_obs + K*w_R_obs + V_L_obs - V_S) for i_obs,w_R_obs,V_L_obs,V_S in zip(Yobs[:,0],Yobs[:,1],Yobs[:,2],U[:,0])]
    r_2 = [(i_obs - i) for i_obs,i in zip(Yobs[:,0],X[:,0])]
    r_3 = [(w_R_obs - w_R) for w_R_obs,w_R in zip(Yobs[:,1],X[:,1])]
    
    f, axarr = plt.subplots(3, sharex=True, figsize=(10,6))
    
    axarr[0].plot(t, r_1, 'b')
    axarr[0].grid(True)
    axarr[0].set_ylabel('$r_{1}(t)$',fontsize=20)

    axarr[1].plot(t, r_2, 'r')
    axarr[1].grid(True)
    axarr[1].set_ylabel('$r_{2}(t)$',fontsize=20)
    
    axarr[2].plot(t, r_3, 'g')
    axarr[2].grid(True)
    axarr[2].set_ylabel('$r_{3}(t)$',fontsize=20)
    
    plt.xlabel('Time (s)',fontsize=16)
    
    f.subplots_adjust(hspace=0.3)
    
    plt.show()